In [2]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import subprocess
# subprocess.run(["pip", "install", "wikipedia-api"])
import wikipediaapi
from gensim.models import Word2Vec

# Initializing the Wikipedia API
wiki = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI, headers={'User-Agent': 'MyUserAgent/1.0'})

#List wikipeida pages here
page_titles = ["Artificial intelligence", "Machine learning", "Data science", "Computers", "Programming languages", "Algorithms", "Robotics", "Computer vision", "Natural language processing", "Deep learning",
               "Computer programming", "Neural networks", "Big data", "Cybersecurity", "Internet of things", "Cloud computing", "Operating systems", "Database management", "Software engineering", "Web development",
               "Information technology", "Computer graphics", "Game development", "Bioinformatics", "Cryptography", "Computer architecture", "Network security", "Computer networks", "Data analysis", "Information retrieval",
               "Parallel computing", "Quantum computing", "Virtual reality", "Augmented reality", "Machine translation", "Speech recognition", "Image processing", "Pattern recognition", "Human-computer interaction", "Computer-assisted design",
               "Distributed computing", "Mobile computing", "Embedded systems", "Computer forensics", "Computer simulation", "Data mining", "Evolutionary computation", "Genetic algorithms", "Data visualization"]

# Fetch content from the above Wikipedia pages
wiki_content = ""
for title in page_titles:
    page = wiki.page(title)
    wiki_content += page.text

print("Wikipedia Content Length:", len(wiki_content))
#tokenization
dictionary_tokens = word_tokenize(wiki_content.lower())  # Convert to lowercase

# Remove numbers and non-alphabetic tokens
dictionary_tokens = [token for token in dictionary_tokens if token.isalpha()]


#Reducing vocab size here
vocab_size = 500000 # 500k words
word_freq = nltk.FreqDist(dictionary_tokens)
top_words = [word for word, _ in word_freq.most_common(vocab_size)]
training_data = [word if word in top_words else "<UNK>" for word in dictionary_tokens]

#Converting tokens into sentences
sentences = [[word] for word in training_data]

#Training the Word2Vec model with the following hyperparams
vector_size = 20
window = 2
min_count = 1
workers = 4
sg = 1
epochs = 100

model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, workers=workers, sg=sg)

for epoch in range(epochs):
    print(f"Training epoch {epoch + 1}/{epochs}")
    model.build_vocab(corpus_iterable=sentences, update=(epoch != 0))
    model.train(corpus_iterable=sentences, total_examples=len(sentences), epochs=1)

# Generate context-target pairs for evaluation
def generate_context_target_pairs(tokens, window_size=2):
    pairs = []
    for i, target_word in enumerate(tokens):
        if(i % 100000 == 0):
            print("Processing token", i, "out of", len(tokens))
        for j in range(max(0, i - window_size), min(len(tokens), i + window_size + 1)):
            if i != j:
                context_word = tokens[j]
                pairs.append((context_word, target_word))
    return pairs

#Example output
random_word = "Artificial" #here we initialize with a random word. using page title for simplicity


num_words = 20
sequence = [random_word]
for i in range(num_words): 
    last_word = sequence[-1]
    predicted_contexts = [predicted[0] for predicted in model.wv.most_similar(last_word)]
    next_word = np.random.choice(predicted_contexts)
    sequence.append(next_word)

#Printing this sequence separated with spaces
print("Generated Sequence:")
print(" ".join(sequence))


Wikipedia Content Length: 1906253
Training epoch 1/100
Training epoch 2/100
Training epoch 3/100
Training epoch 4/100
Training epoch 5/100
Training epoch 6/100
Training epoch 7/100
Training epoch 8/100
Training epoch 9/100
Training epoch 10/100
Training epoch 11/100
Training epoch 12/100
Training epoch 13/100
Training epoch 14/100
Training epoch 15/100
Training epoch 16/100
Training epoch 17/100
Training epoch 18/100
Training epoch 19/100
Training epoch 20/100
Training epoch 21/100
Training epoch 22/100
Training epoch 23/100
Training epoch 24/100
Training epoch 25/100
Training epoch 26/100
Training epoch 27/100
Training epoch 28/100
Training epoch 29/100
Training epoch 30/100
Training epoch 31/100
Training epoch 32/100
Training epoch 33/100
Training epoch 34/100
Training epoch 35/100
Training epoch 36/100
Training epoch 37/100
Training epoch 38/100
Training epoch 39/100
Training epoch 40/100
Training epoch 41/100
Training epoch 42/100
Training epoch 43/100
Training epoch 44/100
Trainin

KeyError: "Key 'Artificial' not present in vocabulary"

In [4]:
# Example output
def generate_sequence(random_word, num_words):
    sequence = [random_word]
    for i in range(num_words): 
        last_word = sequence[-1]
        predicted_contexts = [predicted[0] for predicted in model.wv.most_similar(last_word)]
        next_word = np.random.choice(predicted_contexts)
        sequence.append(next_word)
    return sequence

#Generating a sequence of 20 words
random_word = "artificial"
sequence = generate_sequence(random_word, 20)
print("Generated Sequence:")
print(" ".join(sequence))

for i in range(10):
    random_word = np.random.choice(top_words)
    sequence = generate_sequence(random_word, 20)
    print("Generated Sequence:")
    print(" ".join(sequence))

Generated Sequence:
artificial music deductions rogue animators propofol aborted stereoscopic nuevo trace vista pod viewer characterizations analyzer mistakes winter viral acls viral acls
Generated Sequence:
waveguides hospi cincom warren medium hostile marcian psychosis mayo dilemmas superstructure bitlocker precautions employment reassigned mayo floors manager spirits tend spirits
Generated Sequence:
exposure controller years increased briefly are hockey attempted intent gray spacewar illustrate transforms confirmatory critically joints assignments joints lossy alongside constructing
Generated Sequence:
zadeh terminating ut haiku beetles tobias sweetgreen outputs initial amongst surrogate amongst initial address leveled nikita brookman jwt metcalfe distributors computationalism
Generated Sequence:
fielding bethke threatens dynamics include mermin cables mermin boundaries relations transferring brake musée enterprising theoretically musée msa revitalized missiles bar sebastiaan
Genera